In [2]:
!pip3 install ensemble_boxes

     |████████████████████████████████| 3.3 MB 46.7 MB/s eta 0:00:01
     |████████████████████████████████| 34.5 MB 40.3 MB/s eta 0:00:01


In [3]:
import pandas as pd
from ensemble_boxes import *
import numpy as np
from pycocotools.coco import COCO

In [12]:
# ensemble csv files

submission_files = ["/opt/ml/detection/baseline/mmdetection/output/output (26).csv",
                    "/opt/ml/detection/baseline/mmdetection/output/output (27).csv",
                    "/opt/ml/detection/baseline/mmdetection/output/output (28).csv",
                    # "/opt/ml/detection/baseline/mmdetection/output/output (29).csv",
                    # "/opt/ml/detection/baseline/mmdetection/output/output (30).csv"
                   ]

submission_df = [pd.read_csv(file) for file in submission_files]

In [13]:
image_ids = [f'test/{str(idx).zfill(4)}.jpg' for idx in range(4871)]# submission_df[0]['image_id'].tolist()

In [14]:
annotation = '/opt/ml/detection/dataset/test.json'
coco = COCO(annotation)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [16]:
from tqdm import tqdm
prediction_strings = []
file_names = []
iou_thr = 0.6
skip_box_thr=0
conf_type = 'avg' # 'avg', 'max', 'box_and_model_avg', 'absent_model_aware_avg'
'''
param conf_type: how to calculate confidence in weighted boxes.
    'avg': average value,
    'max': maximum value,
    'box_and_model_avg': box and model wise hybrid weighted average,
    'absent_model_aware_avg': weighted average that takes into account the absent model.
'''

for i, image_id in enumerate(tqdm(image_ids)):
    prediction_string = ''
    boxes_list = []
    scores_list = []
    labels_list = []
    image_info = coco.loadImgs(i)[0]
    
    for df in submission_df:
        if df[df['image_id'] == image_id]['PredictionString'].tolist():
            predict_string = df[df['image_id'] == image_id]['PredictionString'].tolist()[0]
            predict_list = str(predict_string).split()
        
        if len(predict_list)==0 or len(predict_list)==1:
            continue
            
        predict_list = np.reshape(predict_list, (-1, 6))
        box_list = []
        
        for box in predict_list[:, 2:6].tolist():
            box[0] = float(box[0]) / image_info['width']
            box[1] = float(box[1]) / image_info['height']
            box[2] = float(box[2]) / image_info['width']
            box[3] = float(box[3]) / image_info['height']
            box_list.append(box)
            
        boxes_list.append(box_list)
        scores_list.append(list(map(float, predict_list[:, 1].tolist())))
        labels_list.append(list(map(int, predict_list[:, 0].tolist())))
    
    if len(boxes_list):
        boxes, scores, labels = nms(boxes_list, scores_list, labels_list,iou_thr=iou_thr)
        # boxes, scores, labels = soft_nms(box_list, scores_list, labels_list, iou_thr=iou_thr)
        # boxes, scores, labels = non_maximum_weighted(boxes_list, scores_list, labels_list,iou_thr=iou_thr)
        # boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list,iou_thr=0.5,conf_type='box_and_model_avg')
        # boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list, iou_thr=iou_thr, skip_box_thr=skip_box_thr)

        for box, score, label in zip(boxes, scores, labels):
            prediction_string += str(int(label)) + ' ' + str(score) + ' ' + str(box[0] * image_info['width']) + ' ' + str(box[1] * image_info['height']) + ' ' + str(box[2] * image_info['width']) + ' ' + str(box[3] * image_info['height']) + ' '
    
    prediction_strings.append(prediction_string)
    file_names.append(image_id)

  0%|          | 13/4871 [00:03<13:49,  5.85it/s] 

Warning. Fixed 6 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 4 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 2 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]


  1%|          | 31/4871 [00:03<04:05, 19.68it/s]

Warning. Fixed 4 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 1 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 2 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]


  1%|          | 43/4871 [00:04<02:37, 30.74it/s]

...

Warning. Fixed 5 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 1 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 1 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 3 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]


100%|█████████▉| 4861/4871 [01:28<00:00, 55.70it/s]

Warning. Fixed 2 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 3 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 1 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]


100%|██████████| 4871/4871 [01:28<00:00, 55.18it/s]

Warning. Fixed 8 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 1 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]


In [17]:
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv('/opt/ml/detection/baseline/mmdetection/output/ensemble_top5_ensemble.csv', index = False)

submission

,PredictionString,image_id
0,0 0.10780899971723557 327.4179382324219 639.33...,test/0000.jpg
1,0 0.2398233711719513 637.3775024414062 650.397...,test/0001.jpg
2,0 0.7020535469055176 878.8914184570312 463.548...,test/0002.jpg
3,0 0.03331926700315977 15.871737480163574 114.5...,test/0003.jpg
4,0 0.4408690631389618 433.6463317871094 405.499...,test/0004.jpg
...,...,...
4866,0 0.5759724742836423 293.30999755859375 349.20...,test/4866.jpg
4867,0 0.42011648416519165 625.5298461914062 665.24...,test/4867.jpg
4868,0 0.15808932979901633 545.0625610351562 536.87...,test/4868.jpg
4869,0 0.8391531109809875 5.799160480499268 235.651...,test/4869.jpg


In [ ]:
import matplotlib.pyplot as plt
import cv2

index = 190
img = plt.imread(f"/opt/ml/detection/dataset/test/{str(index).zfill(4)}.jpg")

elements = submission.PredictionString[index].split()
i=0
while i<len(elements):
    label, score, x1, y1, x2, y2 = elements[i:i+6]
    if float(score) < 0.3:
        i+=6
        continue
    x1 = int(float(x1))
    y1 = int(float(y1))
    x2 = int(float(x2))
    y2 = int(float(y2))
    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 3)
    i += 6

plt.figure(figsize=(15,15))
plt.imshow(img)